<a href="https://colab.research.google.com/github/eTcilopp/geek_brains/blob/main/044_machine_learning/hw09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание № 9

Задача — реализовать алгоритм случайного леса для задачи классификации и применить его к набору данных.

Шаги выполнения задания:

1. Импортируйте необходимые библиотеки: numpy, pandas, sklearn.

2. Загрузите набор данных для задачи классификации. Вы можете использовать любой доступный набор данных или выбрать один из популярных, таких как Iris, Wine или MNIST.

3. Проведите предварительную обработку данных, включая масштабирование и разделение на обучающую и тестовую выборки.

4. Реализуйте алгоритм случайного леса. Включите в него функции для построения деревьев решений, выбора случайных признаков и голосования для принятия решений.

5. Обучите вашу модель случайного леса на обучающей выборке.

6. Оцените производительность модели на тестовой выборке, используя метрики классификации, такие как точность, полнота и F1-мера.

7. Проведите сравнение результатов вашей модели со стандартной реализацией случайного леса из библиотеки scikit-learn.

In [1]:
!pip install yfinance -q

In [2]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [3]:
import yfinance as yf
from datetime import datetime, timedelta

В качестве датасета возьмем истортю цен на инвестиционный паевой фонд S&P500 и попробуем предсказать, увеличится ли завтрашняя цена закрытия по сравнению с завтрашней ценой закрытия.

In [4]:
TICKER = 'SPY'
YEARS_HISTORY = 1
start_date = datetime.now() - timedelta(days=(365 * YEARS_HISTORY))
df = yf.download(TICKER, start=start_date, end=datetime.now())

[*********************100%***********************]  1 of 1 completed


In [5]:
df.head(2)

Price,Adj Close,Close,High,Low,Open,Volume
Ticker,SPY,SPY,SPY,SPY,SPY,SPY
Date,,,,,,
2023-12-19,470.404144,474.839996,474.920013,472.450012,472.529999,55761800
2023-12-20,463.885590,468.260010,475.899994,467.820007,473.959991,102921000


Добавим Target - он у нас известне вплоть до "вчера"

In [6]:
df['Open Price Next Day'] = df['Open'].shift(-1)
df['Close Price Next Day'] = df['Close'].shift(-1)
df['Target'] = np.where(df['Close Price Next Day'] > df['Open Price Next Day'], 1, 0)

In [7]:
df.head(2)

Price,Adj Close,Close,High,Low,Open,Volume,Open Price Next Day,Close Price Next Day,Target
Ticker,SPY,SPY,SPY,SPY,SPY,SPY,,,
Date,,,,,,,,,
2023-12-19,470.404144,474.839996,474.920013,472.450012,472.529999,55761800,473.959991,468.260010,0
2023-12-20,463.885590,468.260010,475.899994,467.820007,473.959991,102921000,471.329987,472.700012,1


Такой анализ особого смысла в финансовом плане не имеет <br>
(по крайней мере, при использовании Случайного Леса),<br>
 поскольну не учитывает историю движения цены.<br>
 Но мы можем добавить чуть больше информации об этом движении: <br>
 добавим скользящие средние значения цен за 50 и за 200 дней.

In [8]:
df['Moving Average 50'] = df['Close'].rolling(window=50).mean()
df['Moving Average 200'] = df['Close'].rolling(window=200).mean()

Преимуществом такого датасета является его "чистота": можно быть уверенным, что все данные корректны и присутствуют.

In [9]:
df = df.dropna()

Преобразуем даты в дополнительный набор параметров: <br>
номер дня недели<br>
номер месяца<br>
год

In [10]:
df = df.reset_index()
df['Day of the week'] = df['Date'].dt.day_of_week
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year

df = df.drop(columns=['Date'], level=0)

df.head()

Price,Adj Close,Close,High,Low,Open,Volume,Open Price Next Day,Close Price Next Day,Target,Moving Average 50,Moving Average 200,Day of the week,Month,Year
Ticker,SPY,SPY,SPY,SPY,SPY,SPY,,,,,,,,
0,572.979980,572.979980,573.359985,568.099976,572.349976,42939100,571.299988,567.799988,0,554.064597,524.009200,4,10,2024
1,567.799988,567.799988,571.960022,566.630005,571.299988,49964700,570.419983,573.169983,1,554.531797,524.474000,0,10,2024
2,573.169983,573.169983,573.780029,569.530029,570.419983,37398700,573.159973,577.140015,1,555.099996,524.998549,1,10,2024
3,577.140015,577.140015,577.710022,572.549988,573.159973,37912200,575.770020,576.130005,1,555.802797,525.520750,2,10,2024
4,576.130005,576.130005,577.580017,574.489990,575.770020,44138100,576.049988,579.580017,1,556.309197,526.033150,3,10,2024


Отмасштабируем данные

In [11]:
s_columns = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Open Price Next Day', 'Close Price Next Day', 'Moving Average 50', 'Moving Average 200']
scaler = StandardScaler()
df[s_columns] = scaler.fit_transform(df[s_columns])

Получим модель и проверим ее на тестовых данных

In [12]:
X = df.drop(columns=['Target'])
y = df['Target']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)

In [15]:
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

In [16]:
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Accuracy: {accuracy}")

Precision: 1.0
Recall: 0.75
F1 Score: 0.8571428571428571
Accuracy: 0.8181818181818182


Удивительно, но лишь на остовании данных о цене за один день (плюс скользящие средние) мы получили модель, стопроцентно "попадающая" в цель, если предсказывает рост цены "завтра" (Precision 100%), при этом пропуская только 25% кейсов (Recall 75%)

Попробуем реализовать случайный лес вручную


In [64]:
def generate_bootstrap_samples(data, n_samples):
    bootstrap_samples = []
    n_rows = data.shape[0]

    for i in range(n_samples):
        sample_indices = np.random.choice(n_rows, size=n_rows, replace=True)
        sample = data.iloc[sample_indices] if isinstance(data, pd.DataFrame) else data[sample_indices]
        bootstrap_samples.append(sample)

    return bootstrap_samples

In [65]:
def get_predictions(bootstrap_samples, df_test):
  predictions = []
  X_test = df_test.drop(columns=['Target'])
  for sample in bootstrap_samples:
    X_train = sample.drop(columns=['Target'])
    y_train = sample['Target']
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_model.fit(X_train, y_train)
    y_pred = rf_model.predict(X_test)
    predictions.append(y_pred)
  return predictions

In [66]:
def get_most_frequent_value(predictions):
  df = pd.DataFrame(predictions)
  most_frequent = df.mode().iloc[0]
  return most_frequent

In [72]:
df_test, df_train = train_test_split(df, test_size=0.2, random_state=42)
s_columns = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Open Price Next Day', 'Close Price Next Day', 'Moving Average 50', 'Moving Average 200']
scaler = StandardScaler()
df[s_columns] = scaler.fit_transform(df[s_columns])

n_samples = 10  # Количество подвыборок
bootstrap_samples = generate_bootstrap_samples(df_train, n_samples)

In [73]:
predictions = get_predictions(bootstrap_samples, df_test)

In [74]:
y_test_pred = get_most_frequent_value(predictions)

In [75]:
precision = precision_score(df_test['Target'], y_test_pred)
recall = recall_score(df_test['Target'], y_test_pred)
f1 = f1_score(df_test['Target'], y_test_pred)
accuracy = accuracy_score(df_test['Target'], y_test_pred)

In [76]:
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Accuracy: {accuracy}")

Precision: 0.4827586206896552
Recall: 0.7368421052631579
F1 Score: 0.5833333333333334
Accuracy: 0.5


Результаты ухудшились, но, я полагаю, их можно улучшить побрирая гиперпараметры

# Выводы
Случайный лес - передовая методика классификации, позволяющая решить проблему переобучения и получить хорошие результаты на тестовых данных.